# Polymarket Win Rate Analysis

Analyzing calibration and systematic mispricing in Polymarket prediction markets.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from pathlib import Path

# Plot settings
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 7)
plt.rcParams['font.size'] = 11

## Load Data

In [ ]:
# Find and load the most recent data file
data_dir = Path("polymarket_data")
data_files = list(data_dir.glob("polymarket_trades_*.csv"))

if not data_files:
    print("No data files found! Run polymarket_data_collector.py first.")
else:
    latest_file = max(data_files, key=lambda p: p.stat().st_mtime)
    print(f"Loading: {latest_file}")
    
    df = pd.read_csv(latest_file)
    df['trade_timestamp'] = pd.to_datetime(df['trade_timestamp'])
    print(f"Loaded {len(df):,} trades from {df['condition_id'].nunique():,} markets")

## Data Overview

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# Quick stats
print(f"Date range: {df['trade_timestamp'].min()} to {df['trade_timestamp'].max()}")
print(f"\nSide distribution:")
print(df['side'].value_counts())
print(f"\nOutcome distribution:")
print(df['outcome'].value_counts())
print(f"\nOverall win rate: {df['won'].mean():.2%}")
print(f"Average price: {df['price'].mean():.3f}")

In [ ]:
# Category distribution
df['category'].value_counts().head(15)

In [ ]:
# Trades per market distribution
trades_per_market = df.groupby('condition_id').size()
print(f"Trades per market:")
print(f"  Min: {trades_per_market.min()}, Max: {trades_per_market.max()}, Median: {trades_per_market.median():.0f}")
print(f"  Markets with <10 trades:  {(trades_per_market < 10).sum()}")
print(f"  Markets with <50 trades:  {(trades_per_market < 50).sum()}")
print(f"  Markets with 100+ trades: {(trades_per_market >= 100).sum()}")

## Helper Functions

In [ ]:
def wilson_ci(successes, n, confidence=0.95):
    """Wilson score confidence interval for proportions"""
    if n == 0:
        return 0, 0
    z = stats.norm.ppf((1 + confidence) / 2)
    p = successes / n
    denominator = 1 + z**2 / n
    center = (p + z**2 / (2*n)) / denominator
    margin = z * np.sqrt(p*(1-p)/n + z**2/(4*n**2)) / denominator
    return max(0, center - margin), min(1, center + margin)


def calculate_win_rates(data, price_bins=20, min_samples=30):
    """Calculate win rate by price bucket"""
    data = data.copy()
    data['price_bin'] = pd.cut(data['price'], bins=price_bins, include_lowest=True)
    
    results = []
    for price_bin in data['price_bin'].cat.categories:
        bin_data = data[data['price_bin'] == price_bin]
        if len(bin_data) < min_samples:
            continue
        
        n = len(bin_data)
        wins = bin_data['won'].sum()
        win_rate = wins / n
        ci_low, ci_high = wilson_ci(wins, n)
        
        results.append({
            'price_bin': price_bin,
            'price_midpoint': price_bin.mid,
            'win_rate': win_rate,
            'ci_low': ci_low,
            'ci_high': ci_high,
            'n_trades': n,
            'n_markets': bin_data['condition_id'].nunique()
        })
    
    return pd.DataFrame(results)

## Filter to BUY trades only

For calibration analysis, we focus on BUY trades: "Did what I bought win?"

In [ ]:
buys = df[df['side'] == 'BUY'].copy()
print(f"BUY trades: {len(buys):,} ({len(buys)/len(df):.1%} of all trades)")

---
# Main Calibration Analysis

**Key question:** Do contracts at price X win X% of the time?

- Line above diagonal = underpriced (win more than price suggests)
- Line below diagonal = overpriced (win less than price suggests)

In [ ]:
# Calculate win rates
results = calculate_win_rates(buys, price_bins=20, min_samples=50)
results

In [ ]:
# Main calibration plot
fig, ax = plt.subplots(figsize=(12, 8))

# Win rate line
ax.plot(results['price_midpoint'] * 100, results['win_rate'] * 100, 
        'o-', color='#e31e24', linewidth=2.5, markersize=6, label='Observed Win Rate', zorder=3)

# Confidence interval
ax.fill_between(results['price_midpoint'] * 100, results['ci_low'] * 100, results['ci_high'] * 100,
                alpha=0.3, color='#e31e24', label='95% CI')

# Perfect calibration
ax.plot([0, 100], [0, 100], '--', color='#2ecc71', linewidth=2, label='Perfect Calibration', zorder=2)

ax.set_xlabel('Contract Price (cents)', fontsize=13, fontweight='bold')
ax.set_ylabel('Win Percentage', fontsize=13, fontweight='bold')
ax.set_title('Polymarket Calibration: Win Rate vs Price', fontsize=15, fontweight='bold')
ax.set_xlim(0, 100)
ax.set_ylim(0, 100)
ax.legend(loc='upper left', fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calibration error
cal_error = np.abs(results['win_rate'] - results['price_midpoint']).mean()
print(f"Mean absolute calibration error: {cal_error:.4f} ({cal_error*100:.2f} cents)")

---
# YES vs NO Comparison

Are YES contracts priced differently from NO contracts?

In [ ]:
yes_trades = buys[buys['outcome'] == 'Yes']
no_trades = buys[buys['outcome'] == 'No']

print(f"YES trades: {len(yes_trades):,}")
print(f"NO trades:  {len(no_trades):,}")

In [ ]:
yes_results = calculate_win_rates(yes_trades, price_bins=20, min_samples=30)
no_results = calculate_win_rates(no_trades, price_bins=20, min_samples=30)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Left: Both on same plot
ax1.plot(yes_results['price_midpoint'] * 100, yes_results['win_rate'] * 100,
         'o-', color='#3498db', linewidth=2.5, markersize=6, label='YES')
ax1.fill_between(yes_results['price_midpoint'] * 100, yes_results['ci_low'] * 100, 
                 yes_results['ci_high'] * 100, alpha=0.2, color='#3498db')

ax1.plot(no_results['price_midpoint'] * 100, no_results['win_rate'] * 100,
         'o-', color='#e74c3c', linewidth=2.5, markersize=6, label='NO')
ax1.fill_between(no_results['price_midpoint'] * 100, no_results['ci_low'] * 100,
                 no_results['ci_high'] * 100, alpha=0.2, color='#e74c3c')

ax1.plot([0, 100], [0, 100], '--', color='#2ecc71', linewidth=2, label='Perfect')
ax1.set_xlabel('Price (cents)')
ax1.set_ylabel('Win %')
ax1.set_title('YES vs NO Calibration')
ax1.set_xlim(0, 100)
ax1.set_ylim(0, 100)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Right: Deviation from perfect
yes_dev = (yes_results['win_rate'] - yes_results['price_midpoint']) * 100
no_dev = (no_results['win_rate'] - no_results['price_midpoint']) * 100

ax2.plot(yes_results['price_midpoint'] * 100, yes_dev, 'o-', color='#3498db', linewidth=2.5, label='YES')
ax2.plot(no_results['price_midpoint'] * 100, no_dev, 'o-', color='#e74c3c', linewidth=2.5, label='NO')
ax2.axhline(y=0, color='#2ecc71', linestyle='--', linewidth=2)
ax2.fill_between([0, 100], -5, 5, alpha=0.1, color='gray', label='+-5c range')

ax2.set_xlabel('Price (cents)')
ax2.set_ylabel('Deviation (cents)')
ax2.set_title('Deviation from Perfect Calibration')
ax2.set_xlim(0, 100)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Summary stats
print(f"\nYES: Mean deviation = {yes_dev.mean():+.2f}c, Calibration error = {np.abs(yes_dev).mean():.2f}c")
print(f"NO:  Mean deviation = {no_dev.mean():+.2f}c, Calibration error = {np.abs(no_dev).mean():.2f}c")

---
# Analysis by Time to Resolution

A 60c contract 3 months out is very different from 60c 1 hour before resolution.

In [ ]:
# Define time buckets
time_buckets = [
    (0, 24, '0-24h'),
    (24, 72, '1-3 days'),
    (72, 168, '3-7 days'),
    (168, 672, '1-4 weeks'),
    (672, 2016, '1-3 months'),
    (2016, float('inf'), '3+ months')
]

# Filter to trades with valid time_to_resolution
buys_with_time = buys[buys['time_to_resolution_hours'].notna()].copy()
print(f"Trades with time data: {len(buys_with_time):,}")

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, (low, high, label) in enumerate(time_buckets):
    mask = (buys_with_time['time_to_resolution_hours'] >= low) & (buys_with_time['time_to_resolution_hours'] < high)
    bucket_data = buys_with_time[mask]
    
    ax = axes[idx]
    
    if len(bucket_data) < 100:
        ax.text(0.5, 0.5, f'Insufficient data\n(n={len(bucket_data)})', 
                ha='center', va='center', transform=ax.transAxes)
        ax.set_title(f'{label}')
        continue
    
    results = calculate_win_rates(bucket_data, price_bins=15, min_samples=20)
    
    ax.plot(results['price_midpoint'] * 100, results['win_rate'] * 100,
            'o-', color='#e31e24', linewidth=2, markersize=5)
    ax.fill_between(results['price_midpoint'] * 100, results['ci_low'] * 100,
                    results['ci_high'] * 100, alpha=0.3, color='#e31e24')
    ax.plot([0, 100], [0, 100], '--', color='#2ecc71', linewidth=1.5, alpha=0.7)
    
    ax.set_xlim(0, 100)
    ax.set_ylim(0, 100)
    ax.set_xlabel('Price (c)')
    ax.set_ylabel('Win %')
    ax.set_title(f'{label} (n={len(bucket_data):,})', fontweight='bold')
    ax.grid(True, alpha=0.3)

plt.suptitle('Calibration by Time to Resolution', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

---
# Analysis by Category

In [ ]:
# Top categories by trade count
top_categories = buys['category'].value_counts().head(6).index.tolist()
print("Top categories:", top_categories)

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, category in enumerate(top_categories):
    cat_data = buys[buys['category'] == category]
    ax = axes[idx]
    
    if len(cat_data) < 100:
        ax.text(0.5, 0.5, f'Insufficient data\n(n={len(cat_data)})',
                ha='center', va='center', transform=ax.transAxes)
        ax.set_title(f'{category}')
        continue
    
    results = calculate_win_rates(cat_data, price_bins=15, min_samples=20)
    
    ax.plot(results['price_midpoint'] * 100, results['win_rate'] * 100,
            'o-', color='#e31e24', linewidth=2, markersize=5)
    ax.fill_between(results['price_midpoint'] * 100, results['ci_low'] * 100,
                    results['ci_high'] * 100, alpha=0.3, color='#e31e24')
    ax.plot([0, 100], [0, 100], '--', color='#2ecc71', linewidth=1.5, alpha=0.7)
    
    ax.set_xlim(0, 100)
    ax.set_ylim(0, 100)
    ax.set_xlabel('Price (c)')
    ax.set_ylabel('Win %')
    ax.set_title(f'{category} (n={len(cat_data):,})', fontweight='bold')
    ax.grid(True, alpha=0.3)

plt.suptitle('Calibration by Category', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

---
# Price Distribution

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Price distribution
ax1.hist(buys['price'] * 100, bins=50, color='#3498db', alpha=0.7, edgecolor='black')
ax1.set_xlabel('Price (cents)')
ax1.set_ylabel('Number of trades')
ax1.set_title('Distribution of Trade Prices')
ax1.axvline(x=50, color='red', linestyle='--', alpha=0.5)

# Trade size distribution
ax2.hist(buys['size'].clip(upper=1000), bins=50, color='#2ecc71', alpha=0.7, edgecolor='black')
ax2.set_xlabel('Trade Size ($)')
ax2.set_ylabel('Number of trades')
ax2.set_title('Distribution of Trade Sizes (capped at $1000)')

plt.tight_layout()
plt.show()

print(f"Trade size stats:")
print(f"  Mean: ${buys['size'].mean():.2f}")
print(f"  Median: ${buys['size'].median():.2f}")
print(f"  Max: ${buys['size'].max():.2f}")

---
# Custom Analysis Space

Use this section to explore specific hypotheses.

In [ ]:
# Example: Filter to specific conditions and analyze
# Uncomment and modify as needed

# High volume markets only
# high_vol = buys[buys['volume_total'] > 100000]
# results = calculate_win_rates(high_vol)

# Specific category
# politics = buys[buys['category'] == 'Politics']
# results = calculate_win_rates(politics)

# Recent trades only
# recent = buys[buys['trade_timestamp'] > '2024-06-01']
# results = calculate_win_rates(recent)

---
# Findings & Notes

*Add your observations here as you explore the data.*

- 
- 
- 